### LAFOURCADE Emie p1802635
## **Projet RC3 - Kaggle Challenge**
### Lifprojet Printemps 2021
### Encadrant : CAZABET Remy

### **Description :**
Le projet consiste à découvrir Kaggle et participer à une "compétition" sur cette plateforme.

Sur ce notebook se trouve le code que j'ai réalisé pour la compétition ["Natural Language Processing with Disaster Tweets"](http://www.kaggle.com/c/nlp-getting-started). Il s'agit de déterminer si les tweets à tester sont des tweets parlant d'une catastrophe (disaster) ou non à partir d'une base de données de tweets dont nous connaissons l'issue (disaster ou non disaster).

### **Exécution :**
Pour éxécuter ce code il faut se mettre en "edit mode" et cliquer sur la double flèche "Run all". Cependant, il met une 15ène de minutes à s'éxécuter. Toutes les sorties que vous obtiendrez sont affichées en viewer mode alors il n'est pas nécéssaire de le lancer.
Ce code n'est éxécutable que sur Kaggle (emplacement des données et librairies propre).

D'autres noteboooks sont consultables sur mon profil.
### **Résultats :**
J'ai testé plusieurs solutions et la meilleure reste celle avec le random forest classificateur qui donne un résultat d'environ 79% de réussite.

### **Sommaire :**
*   I. Importation des librairies et données
*   II. Data Visualization
*   III. Traitement des données
*   IV. Exploitations des données
*   V. Resultats

# **I-Importation des librairies et données**

In [ ]:
import re
import pandas as pd
import numpy as np
import os
import random
import seaborn as sns 
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from sklearn import feature_extraction, linear_model, model_selection, preprocessing, metrics,manifold
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
import shelve
from wordcloud import WordCloud 
import spacy 
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
train_df=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

# **II-Data Visualization**

## **II.1-Forme des données**

In [ ]:
train_df.head()

In [ ]:
train_df.axes

#### train_df est composé de 5 colonnes : id, keyword, location, text et target et 7613 lignes

In [ ]:
test_df.head()

In [ ]:
test_df.axes

#### test_df est composé de 4 colonnes : id, keyword, location et text et 3263 lignes

## **II.2-Eléments nuls**

In [ ]:
nulls = pd.DataFrame({"Num_Null": train_df.isnull().sum()})
nulls["Pct_Null"] = nulls["Num_Null"] / train_df.count() * 100
nulls

#### Valeurs nulles dans train_df

In [ ]:
nulls = pd.DataFrame({"Num_Null": test_df.isnull().sum()})
nulls["Pct_Null"] = nulls["Num_Null"] / test_df.count() * 100
nulls

#### Valeurs nulles dans test_df

#### Environ la moitier des tweets totaux n'indique pas de localisation. La colonne "location" ne sera donc pas utilisée. 

#### On pourra cependant utiliser la colonne des mots clés "keywords".

## **II.3-Taux des tweets disaster - non disaster**

In [ ]:
target_vc = train_df["target"].value_counts(normalize=True)
print("Non Disaster: {:.2%}, Disaster: {:.2%}".format(target_vc[0], target_vc[1]))
sns.barplot(x=target_vc.index, y=target_vc)
plt.title("Histogramme Disaster vs Non-Disaster")
plt.xlabel("0 = Non-Disaster, 1 = Disaster")
plt.show()

## **II.4-Longueurs des tweets**

In [ ]:
train_df["tweet_length"] = train_df["text"].apply(len)
g = sns.FacetGrid(train_df, col="target", height=5)
g = g.map(sns.histplot, "tweet_length")
plt.suptitle("Distribution Tweet Length")
plt.show()

#### La longueur des tweet n'est pas déterminante dans notre recherche de "disaster tweets".

## **II.5-Keywords les plus utilisés**

In [ ]:
keywords_vc = pd.DataFrame({"Count": (train_df["keyword"].append(test_df["keyword"])).value_counts()})
sns.set()
plt.figure(figsize = (10,40))
sns.barplot(y=keywords_vc[0:300].index, x=keywords_vc[0:300]["Count"])
plt.title("Keywords les plus utilisés", fontsize = 20)
plt.xlabel("nb occurences", fontsize = 15)
plt.ylabel("Keywords",fontsize = 15)
plt.show()

In [ ]:
disaster_keywords = train_df.loc[train_df["target"] == 1]["keyword"].value_counts()
nondisaster_keywords = train_df.loc[train_df["target"] == 0]["keyword"].value_counts()

fig, ax = plt.subplots(1,2, figsize=(20,8))
sns.barplot(y=disaster_keywords[0:30].index, x=disaster_keywords[0:30], orient='h', ax=ax[0], palette="Reds_d")
sns.barplot(y=nondisaster_keywords[0:30].index, x=nondisaster_keywords[0:30], orient='h', ax=ax[1], palette="Blues_d")
ax[0].set_title("Keywords les plus utilisés dans les Disaster Tweets",fontsize = 25)
ax[0].set_xlabel("nb occurences",fontsize = 20)
ax[1].set_title("Keywords les plus utilisés dans les Non-Disaster Tweets",fontsize = 25)
ax[1].set_xlabel("nb occurences",fontsize = 20)
plt.tight_layout()
plt.show()

In [ ]:
#Dictionnaire avec les keywords de train_df et le nombre de fois qu'ils apparaissent dans train_df
keywords_proba_df = pd.DataFrame({"keyword":train_df["keyword"].value_counts().index,"Count": (train_df["keyword"].value_counts().values)})

In [ ]:
def proba_keywords(n, df=keywords_proba_df["keyword"]):    
    """
    Arguments :
    :n: integer
    :df: liste de keyword
    :return: [int,float]
    
    Cette fonction renvoie un couple d'int 
    dont le premier indique si le keyword df[n] a été déterminé comme tweet disaster(1) ou non (0), 
    et le second indique la probabilité qu'il soit un tweet disaster ou non.
    """
    tweets_keywords=train_df.loc[train_df["keyword"]==df[n]]
    tot=0
    for i in tweets_keywords.index:
        if tweets_keywords['target'][i]==1:
            tot+=1
    if len(tweets_keywords) >0:proba=tot/len(tweets_keywords)
    else: proba=-1
    return([0, proba] if proba<0.4297 else [-1,proba] if proba==-1 else [1, proba])

In [ ]:
for i in range(len(keywords_proba_df)):
    keywords_proba_df.loc[i, "proba_target"],keywords_proba_df.loc[i, "proba"] =proba_keywords(i)
    
keywords_proba_df.head()

In [ ]:
disaster_keywords = train_df.loc[train_df["target"] == 1]["keyword"].value_counts()

dk_proba = pd.DataFrame({"keyword":disaster_keywords.keys(), "Count" :disaster_keywords.values})

for i in range(len(dk_proba)):
    dk_proba.loc[i, "proba_target"],dk_proba.loc[i, "proba"]=proba_keywords(i,disaster_keywords.keys() )
    
dk_proba

In [ ]:
x = dk_proba["Count"]
y = dk_proba["proba"]
plt.plot(x, y)
plt.xlabel("Nombre d'apparitions du keyword dans train", fontsize=15)
plt.ylabel("Probabilité de disaster", fontsize=15)
plt.show()
plt.close()

#### Les keywords les plus utilisés dans les disaster tweets sont un bon indicateur pour classer les tweets. Un tweet ayant comme keyword un mot très utilisé dans les disaster tweets a de grandes chances d'en être un aussi.

In [ ]:
non_disaster_keywords = train_df.loc[train_df["target"] == 0]["keyword"].value_counts()

ndk_proba = pd.DataFrame({"keyword":non_disaster_keywords.keys(), "Count" :non_disaster_keywords.values})

for i in range(len(ndk_proba)):
    ndk_proba.loc[i, "proba_target"],ndk_proba.loc[i, "proba"]=proba_keywords(i,non_disaster_keywords.keys() )
    
ndk_proba

In [ ]:
x = ndk_proba["Count"]
y = ndk_proba["proba"]
fig, ax = plt.subplots()

plt.plot(x, y)
plt.xlabel("Nombre d'apparitions du keyword dans train", fontsize=15)
plt.ylabel("Probabilité de non disaster", fontsize=15)
ax.invert_yaxis()
plt.show()
plt.close()

#### Même chose pour les non-disaster tweets.

# **III-Traitement des données**

In [ ]:
def cleaning(sentence):
    """
    Arguments :
    :sentence: str
    :return: str 
    
    Cette fonction renvoie la phrase passée en paramètre "nétoyée".
    Elle est passée en minuscule, les abréviations sont remplacées par les mots complets
    et la ponctuation est enlevée.
    """
    sentence = sentence.lower()
    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"gon", "going", sentence)
    sentence = re.sub(r"gonna", "going", sentence)
    sentence = re.sub(r"whats", "what is", sentence)
    sentence = re.sub(r"im", "i am", sentence)
    sentence = re.sub(r"ill", "i will", sentence)
    sentence = re.sub(r"na", "", sentence)
    sentence = re.sub(r"can\'t", "can not", sentence)
    sentence = re.sub(r"cant", "can not", sentence)
    sentence = re.sub(r"cannot", "can not", sentence)
    sentence = re.sub(r"didnt", "did not", sentence)
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence) 
    return sentence

stop_words = set(stopwords.words('english'))  

In [ ]:
stop_words = set(stopwords.words('english')+["co","http"])
snow = nltk.stem.SnowballStemmer('english')

def mots_cles(df):
    """
    Arguments :
    :df: dataframe
    
    Cette fonction ajoute deux colonnes au dataframe passé en paramètre.
    Une colonne "mots_clés" qui correspond au tweet de base dépourvus des mots inutiles (stowords)
    et une colonne "mots_clés_snowball" qui correspond à ces mêmes mots appliqués à la fonction snowballStemmer.
    """
    temp = []
    temp_s=[]

    for each_sentence in df.text:
        each_sentence = cleaning(each_sentence)
        each_word=[]
        each_word_s=[]
        for word in each_sentence.split():
            if word not in stop_words:
                each_word.append(word)
                each_word_s.append(snow.stem(word))
        
        temp.append(each_word)
        temp_s.append(each_word_s) 
        
        final_word = []
        final_word_s=[]

    for row in temp:
        seq = ''
        for word in row:
            seq = seq + word + ' '
        final_word.append(seq)
        
    for row in temp_s:
        seq = ''
        for word in row:
            seq = seq + word + ' '
        final_word_s.append(seq)

    for i in range(len(df)):
        df.loc[i, "mots_cles"]=final_word[i]
        df.loc[i, "mots_cles_snowball"]=final_word_s[i]
    
    return df

In [ ]:
train_df=mots_cles(train_df)
test_df=mots_cles(test_df)

train_df

#### On obtient maintenant les mots utiles de nos tweets dans "mots_cles". On s'est débarrassé des stopwords.

#### La fonction snowball consiste à retirer les pré-fixes et post-fixes des mots pour n'obtenir que les corps de ces derniers. Par exemple "evacuation" et "evacuate" deviennent tous deux "evacuat". Dans le code qui suit, je n'ai pas utilisé ces mots clés car ils donnaient de bien moins bons résultats.

## Visualisation des données traitées

In [ ]:
def to_corpus(target):
    corpus = []

    for w in train_df.loc[train_df["target"] == target]["mots_cles"].str.split():
        for i in w:
            corpus.append(i)
            
    return corpus

def corpus_to_dict(target):
    corpus = to_corpus(target)
            
    dict = defaultdict(int)
    for w in corpus:
        dict[w] += 1
    return sorted(dict.items(), key=lambda x:x[1], reverse=True)

In [ ]:
disaster_dict = corpus_to_dict(0)
non_disaster_dict = corpus_to_dict(1)

disaster_x, disaster_y = zip(*disaster_dict)
non_disaster_x, non_disaster_y = zip(*non_disaster_dict)

fig, ax = plt.subplots(1,2, figsize=(20,8))
sns.barplot(y=list(disaster_x)[0:30], x=list(disaster_y)[0:30], orient='h', palette="Reds_d", ax=ax[0])
sns.barplot(y=list(non_disaster_x)[0:30], x=list(non_disaster_y)[0:30], orient='h', palette="Blues_d", ax=ax[1]) 
ax[0].set_title("mots non stop words les plus utilisés Disaster Tweets",fontsize = 25)
ax[0].set_xlabel("nb occurences",fontsize = 20)
ax[1].set_title("mots non stop words les plus utilisés Non-Disaster Tweets",fontsize = 25)
ax[1].set_xlabel("nb occurences",fontsize = 20)
plt.tight_layout()
plt.show()

In [ ]:
def bigrams(target):
    corpus = train_df[train_df["target"] == target]["mots_cles"]
    count_vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    mots = count_vec.transform(corpus)
    somme_mots = mots.sum(axis=0) 
    freq_mots = [(w, somme_mots[0, idx]) for w, idx in count_vec.vocabulary_.items()]
    freq_mots =sorted(freq_mots, key = lambda x: x[1], reverse=True)
    return freq_mots

In [ ]:
bigrams_disaster = bigrams(1)[:15]
bigrams_non_disaster = bigrams(0)[:15]

x_disaster, y_disaster = map(list, zip(*bigrams_disaster))
x_non_disaster, y_non_disaster = map(list, zip(*bigrams_non_disaster))

fig, ax = plt.subplots(1,2, figsize=(20,8))
sns.barplot(x=y_disaster, y=x_disaster, orient='h', palette="Reds_d", ax=ax[0])
sns.barplot(x=y_non_disaster, y=x_non_disaster, orient='h', palette="Blues_d", ax=ax[1])

ax[0].set_title("Top 15 Bigrams - Disaster Tweets", fontsize=20)
ax[0].set_xlabel("Frequence", fontsize=20)
ax[1].set_title("Top 15 Bigrams - Non-Disaster Tweets", fontsize=20)
ax[1].set_xlabel("Frequence", fontsize=20)
plt.tight_layout()
plt.show()

## Wordclouds

In [ ]:
def wordcloud(mots, titre):
    wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=130).generate(mots) 
    plt.figure(figsize=(10, 7)) 
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off') 
    print(titre)
    plt.show()

In [ ]:
wordcloud(' '.join([text for text in (train_df["mots_cles"].append(test_df["mots_cles"]))]), "Mots principaux tweets totaux")

In [ ]:
wordcloud(' '.join([text for text in train_df['mots_cles'][train_df['target'] == 1]]) , "Mots principaux train tweets disaster")

In [ ]:
wordcloud(' '.join([text for text in train_df['mots_cles'][train_df['target'] == 0]]) , "Mots principaux train tweets non disaster")

# **IV-Exploitation des données** 

## IV.1-Fonctions utiles 

In [ ]:
def conf_matrix(pred, titre):
    """
    Arguments :
    :pred: int[]
    :titre: str
    
    Cette fonction crée une matrice de confusion entre des prédictions passées en parametre et les prédictions officielles de train_df"""
    cmatrix = confusion_matrix(train_df["target"],pred)
    f,ax = plt.subplots(figsize=(3,3))
    sns.heatmap(cmatrix,annot=True,linewidths=0.5,cbar=False,fmt='.0f',ax=ax)
    plt.xlabel("y_predict")
    plt.ylabel("y_true")
    ax.set(title=titre)
    plt.show()

In [ ]:
#Exemple de matrice de confusion avec les valeurs réelles des tweets
conf_matrix(train_df["target"],"Exemple")

#### **Lire une matrice de confusion :**
#### La case en haut à gauche indique le nombre de tweets que le modèle a estimé à 0 et qui valent réellement 0
#### La case en haut à droite indique le nombre de tweets que le modèle a estimé à 1 et qui valent réellement 0
#### La case en bas à gauche indique le nombre de tweets que le modèle a estimé à 0 et qui valent réellement 1
#### La case en bas à droite indique le nombre de tweets que le modèle a estimé à 1 et qui valent réellement 1

In [ ]:
compare_model=[]
#tableau de dictionnaires qui va nous permettre de comparer les différente façons de calculer les tweet disaster 

In [ ]:
def score(pred):
    """
    Arguments :
    :pred: int[]
    
    Calcule le taux de réussite des prédictions"""
    tot=0
    for i in range(len(train_df)):
        if pred[i]==train_df["target"][i]:
            tot+=1
    s=tot/len(train_df)
    print ("Score Train -->", round(s *100,2), " %")
    return s

## **IV.2 Solutions** 
## **Solution 1 : Keywords**

#### On réutilise la fonction proba_keywords définie au dessus pour déterminer si les tweets à tester sont des disaster ou non selon la moyenne des résultats des train tweets partageant le même keyword.

In [ ]:
keywords_proba_df.head()

In [ ]:
def prediction_keywords(i,df=test_df, affichage=False):
    """
    Arguments :
    :i: int
    :df: dataframe
    :affichage: booleen
    
    Determine si le tweet est disaster ou non grâce au tableau keywords_proba_df"""
    if affichage :
        print ("tweet testé :",df.text[i])
        print("keyword :",df.keyword[i])
        if df.text[i]==train_df.text[i]:
            print("target :",df.target[i],"\n")
            
    if df.keyword[i] in list(keywords_proba_df["keyword"]):
        pred=round(keywords_proba_df.proba_target[keywords_proba_df.keyword==df.keyword[i]].values[0])
#   else :pred=(random.randint(0, 1)) #score : 73.74
#   else :pred=9 #score : 73.74
    else :pred=1 #score : 74.04
    if affichage : print ("test :",pred,"--> tweet non disaster" if pred==0 else " --> tweet disaster")
    else: return pred
        

#### Plusieurs tweets ne possèdent pas de keywords. Dans ce cas on met ces tweets à 1 (disaster) car cela donne un meilleur score quand on teste cette technique sur les train tweets.

In [ ]:
prediction_keywords(508,train_df, True)

In [ ]:
prediction_keywords(509,train_df, True)

In [ ]:
prediction_keywords(66,affichage =True)

In [ ]:
predictions_train_keywords=[]
for i in range(len(train_df)):
    predictions_train_keywords.append(prediction_keywords(i,train_df))

In [ ]:
conf_matrix(predictions_train_keywords, "pred keywords")

In [ ]:
score_keywords=score(predictions_train_keywords)

In [ ]:
predictions_keywords=[]
for i in range(len(test_df)):
    predictions_keywords.append(prediction_keywords(i))

In [ ]:
pred={"model":"keywords()", "predictions":np.array(predictions_keywords), "train_predictions":np.array(predictions_train_keywords), "score":score_keywords, "tfidf":False}
compare_model.append(pred)

## **Solution 2 : Vecteurs, Word Embedding**
#### On crée les vecteurs des tweets puis on regarde quels sont les vecteurs des train tweets les plus proches pour chaque tweet à tester. Selon la moyenne des "target" de ces train tweets on détermine si le tweet est un disaster ou non.
### Création des vecteurs 

In [ ]:
nlp = spacy.load("en_core_web_lg")

def valeurTweet(dct):
    """
    Arguments :
    :dct: dictionnary
    :return:liste de taille 300 de float
    
    Crée les vecteurs de chaque mots du dictionnaire avec Spacy (word embedding)"""
    vec = []
    for i in range(len(dct)):
        valTweet={}
        for token in dct[i]["Mots"]:
            valTweet[str(token)] = nlp(token).vector
        vec.append(valTweet)
    return vec

In [ ]:
#Calcul des poids tfidf de chaque mots de la data
model_tfidf=TfidfVectorizer()
tfidf_general = model_tfidf.fit_transform(list(train_df["mots_cles"])+list(test_df["mots_cles"]))

In [ ]:
def dictionnaire(df, tfidf, decalage=0):
    """
    Arguments :
    :df: dataframe
    :tfidf: matrice tfidf
    :decalage: int
    :return: dictionnary
    
    Crée le dictionnaire des données nécéssaires par la suite """
    dct=[]
    for i in df.index:
        dct.append(
            {"id": df.id[i]}) 
    
    for i in df.index:
        dct[i]["Mots"]=[df["mots_cles"][i].split()[j] for j in range(len(df["mots_cles"][i].split())) if df["mots_cles"][i].split()[j] in model_tfidf.vocabulary_]
        
    for i in range(len(dct)):
        dct[i]["index"]=[model_tfidf.vocabulary_[str(k)] for k in dct[i]["Mots"] if str(k) in model_tfidf.vocabulary_ ]
        
    for i in range(len(dct)):
        dct[i]["poids_tfidf"]=[(tfidf[i+decalage, k]) for k in dct[i]["index"]]
        
    vec=valeurTweet(dct)
    for i in range(len(dct)):
        dct[i]["vectors"]=[]
        dct[i]["vectors_tfidf"]=[]
        for k in range(len(dct[i]["Mots"])):
            dct[i]["vectors"].append([0 for l in range(300)])
            dct[i]["vectors_tfidf"].append([0 for l in range(300)])
        for j in range(len(dct[i]["Mots"])):
            #print(dct[i]["Mots"][j], i)
            dct[i]["vectors"][j]=vec[i][dct[i]["Mots"][j]]
            dct[i]["vectors_tfidf"][j]=dct[i]["vectors"][j]*dct[i]["poids_tfidf"][j]
            
    for i in range(len(dct)):
        if dct[i]["Mots"]==[]:
            dct[i]["moy_vector"]=[0 for l in range(300)]
            dct[i]["moy_vector_tfidf"]=[0 for l in range(300)]
        else : 
            dct[i]["moy_vector"]=np.mean([(dct[i]["vectors"][j]) for j in range(len(dct[i]["vectors"]))], axis=0)
            dct[i]["moy_vector_tfidf"]=np.mean([(dct[i]["vectors_tfidf"][j]) for j in range(len(dct[i]["vectors_tfidf"]))], axis=0)
            
    return dct

In [ ]:
# import json

# dct_train=dictionnaire(train_df, tfidf_general)
# dct_test=dictionnaire(test_df, tfidf_general, len(train_df))

# def list_to_file(list_dct, nomFichier):
#     shelf = shelve.open(nomFichier)
#     shelf["my_dict"] = list_dct
#     shelf.close()
#     return shelf 

# shelf_train=list_to_file(dct_train, "list_dct_train.shlf")
# shelf_test=list_to_file(dct_test, "list_dct_test.shlf")

#### On ne lance pas cette cellule car elle met beaucoup de temps à s'éxécuter. Elle permet la création de nos dictionnaires et l'écriture de ceux-ci dans des fichiers shelves. On récupère les données de ces fichiers dans la cellule suivante.

In [ ]:
!cp -r ../input/dictionnaries ./

shelf_train = shelve.open("./dictionnaries/list_dct_train.shlf")
dct_train = shelf_train["my_dict"]
shelf_train.close()
shelf_test = shelve.open("./dictionnaries/list_dct_test.shlf")
dct_test = shelf_test["my_dict"]
shelf_test.close()

In [ ]:
print("id :",dct_train[0]['id'])
print("Mots :",dct_train[0]['Mots'])
print("index :",dct_train[0]['index'])
print("poids_tfidf :",dct_train[0]['poids_tfidf'])
print("vectors :",dct_train[0]['vectors'][0][:5], "etc...")
print("vectors tfidf:",dct_train[0]['vectors_tfidf'][0][:5], "etc...")
print("moy vector:",dct_train[0]['moy_vector'][:5], "etc...")
print("moy vector tfidf:",dct_train[0]['moy_vector_tfidf'][:5], "etc...")

#### Pour chaque tweet, on a son id dans son dataframe, les mots importants qui le composent, son numero d'index dans le dictionnaire tfidf, son poids tfidf, un vecteur de taille 300 pour chaque mot du tweet, ce même vecteur multiplié par le poids tfidf du mot, un vecteur de taille 300 faisant la moyenne des vecteurs de chaque mots et un vecteur de taille 300 faisant la moyenne pondérée avec tfidf des vecteurs de chaque mot.

In [ ]:
X=np.array([dct_train[i]["moy_vector"] for i in range(len(dct_train))]) #Tableau des vecteurs par tweets train
X_test=np.array([dct_test[i]["moy_vector"] for i in range(len(dct_test))]) #Tableau des vecteurs par tweets test

X_tfidf=np.array([dct_train[i]["moy_vector_tfidf"] for i in range(len(dct_train))]) #Tableau des vecteurs tfidf par tweets train
X_test_tfidf=np.array([dct_test[i]["moy_vector_tfidf"] for i in range(len(dct_test))]) #Tableau des vecteurs tfidf par tweets test

y = train_df["target"] #Tableau des target de train

In [ ]:
modele_tsne = manifold.TSNE(2)
def tsne (x=X, y_tsne=y):
    """
    Arguments :
    :x:int[]
    :y_tsne:int[]
    :return: figure tsne
    
    Crée et affiche le modèle tsne de x y_tsne"""
    modele_tsne_fit=modele_tsne.fit_transform(x)

    df_plot=pd.DataFrame(modele_tsne_fit)
    df_plot['target']=y_tsne
    df_plot.columns=["x","y",'target']
    return sns.scatterplot(data=df_plot, x='x', y='y', hue='target')

#### Le modèle tsne permet de rammener les vecteurs de taille 300 à une taille 2 ce qui permet de visualiser si les vecteurs des tweets disaster et non disaster sont distinguables dans un environnement 2D.

In [ ]:
tsne()

#### Les vecteurs sans tfidf forment un nuage de points mais on observe une légère séparation des tweets disaster et non disaster. 

In [ ]:
tsne(x=X_tfidf)

#### Même chose pour les vecteurs pondérés avec tfidf.

### Calculs des distances et vecteurs proches.

In [ ]:
distance_vec=metrics.pairwise_distances(X_test,X) #distance entre les vecteurs de test_df et train_df
distance_vec_train=metrics.pairwise_distances(X,X)#distance entre les vecteurs de train_df
distance_vec_tfidf=metrics.pairwise_distances(X_test_tfidf,X_tfidf)#distance entre les vecteurs tfidf de test_df et train_df
distance_vec_train_tfidf=metrics.pairwise_distances(X_tfidf,X_tfidf)#distance entre les vecteurs tfidf de train_df

In [ ]:
def vecteurs_proches(id_tweet_testé, nb_tweets=100, dist=distance_vec, df=test_df, affichage=False):
    """
    Arguments :
    :id_tweet_testé:int
    :nb_tweets:int
    :dist: float[]
    :df: dataframe
    :affichage: booleen
    :return: int
    
    Calcule si le tweet df[id_tweet_testé] est un disaster(1) ou non(0) 
    grace à la moyenne des nb_tweets target des vecteurs train les plus proches de lui"""
    tab=dist[id_tweet_testé]
    if affichage:
        print ("Tweet testé :",df.text[id_tweet_testé])
        print ("Tweet plus ressemblant :",train_df.text[tab.argsort()[0]])
        print("Target :",train_df.target[tab.argsort()[0]])
    val=[]
    for i in tab.argsort()[0:nb_tweets]:
        val.append(train_df.target[i])
    if affichage: print ("Test :","0 --> tweet non disaster" if np.mean(val)<0.4297else "1 --> tweet disaster")
    return(0 if np.mean(val)<0.4297 else 1)

In [ ]:
r=vecteurs_proches(0, affichage=True)

In [ ]:
r=vecteurs_proches(0, dist=distance_vec_tfidf, affichage=True)

In [ ]:
r=vecteurs_proches(1,affichage=True)

In [ ]:
r=vecteurs_proches(1, dist=distance_vec_tfidf,affichage=True)

In [ ]:
r=vecteurs_proches(0,dist=distance_vec_train, df=train_df,affichage=True)

In [ ]:
predictions_train_vectors=[]
for i in range(len(train_df)):
    predictions_train_vectors.append(vecteurs_proches(i,dist=distance_vec_train, df=train_df))

In [ ]:
conf_matrix(predictions_train_vectors, "100 tweets proches")

In [ ]:
score_vectors=score(predictions_train_vectors)

In [ ]:
predictions_vectors=[]
for i in range(len(test_df)):
    predictions_vectors.append(vecteurs_proches(i))

In [ ]:
pred={"model":"vecteurs_proches()", "predictions":np.array(predictions_vectors), "train_predictions":np.array(predictions_train_vectors), "score":score_vectors, "tfidf":False}
compare_model.append(pred)

In [ ]:
predictions_train_vectors_tfidf=[]
for i in range(len(train_df)):
    predictions_train_vectors_tfidf.append(vecteurs_proches(i,dist=distance_vec_train_tfidf, df=train_df))

In [ ]:
conf_matrix(predictions_train_vectors_tfidf, "100 tweets proches tfidf")

In [ ]:
score_vectors_tfidf=score(predictions_train_vectors_tfidf)

In [ ]:
predictions_vectors_tfidf=[]
for i in range(len(test_df)):
    predictions_vectors_tfidf.append(vecteurs_proches(i,dist=distance_vec_tfidf))

In [ ]:
pred_tfidf={"model":"vecteurs_proches()", "predictions":np.array(predictions_vectors_tfidf), "train_predictions":np.array(predictions_train_vectors_tfidf), "score":score_vectors_tfidf, "tfidf":True}
compare_model.append(pred_tfidf)

## **Solution 3 : Classificateurs**
#### On passe  les vecteurs créés au dessus à différents modèles qui classifient.
#### On teste le Random forest, logistic regression, Gaussian NB, XGboost avec les paramètres minimum puis ces mêmes modèles avec des paramètres déterminés avec grid search et enfin un modèle "voting classifier" avec ces 4 classificateurs.

In [ ]:
def predire(model, titre='', tfidf=False):
    """
    Arguments :
    :model:classifier
    :titree:str
    :tfidf:booleen
    :return: dictionnary
    
    Crée un dictionnaire avec les prédictions sur test_df, les prédictions sur train_df et le score fait avec le modèle model"""
    if tfidf:
        train_X=X_tfidf 
        test_X=X_test_tfidf
    else :
        train_X=X
        test_X=X_test
    predictions = model.predict(test_X)
    train_predictions=model.predict(train_X)
    conf_matrix(train_predictions , titre)
    score=model.score(train_X, y)
    print ("Score Train -->", round(score *100,2), " %")
    pred={"model":str(model), "predictions":predictions, "train_predictions":train_predictions, "score":score, "tfidf":tfidf}
    return pred

In [ ]:
def grid_search(param, base,tfidf=False):
    """
    Arguments :
    :param:parametre du classificateur
    :base:model classifier
    :tfidf:booleen
    :return: model classifier
    
     Recherche du meilleur classificateur grace à une grid search"""
    
    if tfidf:
        train_X=X_tfidf 
        test_X=X_test_tfidf
    else :
        train_X=X
        test_X=X_test
    sh=HalvingGridSearchCV(base, param, cv=5,factor=2, max_resources=50).fit(train_X, y)
    model=sh.best_estimator_
    print(str(model)[:200])
    return model

In [ ]:
pred_lc=predire(LogisticRegression().fit(X,y), "logistic regression")
pred_lc_tfidf=predire(LogisticRegression().fit(X_tfidf,y),"logistic regression tfidf", tfidf=True)

compare_model.append(pred_lc)
compare_model.append(pred_lc_tfidf)

In [ ]:
param_lc = { 'max_iter':[50,100,150,200], 'random_state':[None, 0,1]}
base_lc = LogisticRegression()

pred_lc_gs=predire(grid_search(param_lc, base_lc), "logistic regression grid search")
pred_lc_tfidf_gs=predire(grid_search(param_lc, base_lc, tfidf=True), "logistic regression grid search tfidf",tfidf=True)

compare_model.append(pred_lc_gs)
compare_model.append(pred_lc_tfidf_gs)

In [ ]:
pred_rf=predire(RandomForestClassifier().fit(X,y), "random forest")
pred_rf_tfidf=predire(RandomForestClassifier().fit(X_tfidf,y),"random forest tfidf", tfidf=True)

compare_model.append(pred_rf)
compare_model.append(pred_rf_tfidf)

In [ ]:
param_rf = {'n_estimators':[1,10, 100, 150], 'max_depth':[None, 3,10,50,100], 'random_state':[None, 0,1]}
base_rf = RandomForestClassifier()

pred_rf_gs=predire(grid_search(param_rf, base_rf), "random forest grid search")
pred_rf_tfidf_gs=predire(grid_search(param_rf, base_rf, tfidf=True), "random forest grid search tfidf", tfidf=True)

compare_model.append(pred_rf_gs)
compare_model.append(pred_rf_tfidf_gs)

In [ ]:
pred_gnb=predire(GaussianNB().fit(X,y), "Gaussian NB")
pred_gnb_tfidf=predire(GaussianNB().fit(X_tfidf,y),"Gaussian NB tfidf", tfidf=True)

compare_model.append(pred_gnb)
compare_model.append(pred_gnb_tfidf)

In [ ]:
pred_xgb=predire(XGBClassifier(use_label_encoder=False, eval_metric=["auc"], random_state=0).fit(X,y), "xgboost")
pred_xgb_tfidf=predire(XGBClassifier(use_label_encoder=False, eval_metric=["auc"], random_state=0).fit(X_tfidf,y),"xgboost tfidf", tfidf=True)

compare_model.append(pred_xgb)
compare_model.append(pred_xgb_tfidf)

In [ ]:
param_xgb = {'n_estimators':[1,10, 100, 150], 'random_state':[None, 0,1], 'max_depth':[None, 3,10,50,100]}
base_xgb = XGBClassifier(use_label_encoder=False, eval_metric=["auc"])

pred_xgb_gs=predire(grid_search(param_xgb, base_xgb), "xgboost grid search")
pred_xgb_tfidf_gs=predire(grid_search(param_xgb, base_xgb, tfidf=True), "xgboost grid search tfidf", tfidf=True)

compare_model.append(pred_xgb_gs)
compare_model.append(pred_xgb_tfidf_gs)

In [ ]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()
clf4=XGBClassifier(use_label_encoder=False, eval_metric=["auc"])

def voting_classifier(param='hard', lr=1, rf=1, gnb=1, xgb=1, titre='', tfidf=False):
    """
    Arguments :
    :param:str, parametre du voting classifier ("hard" ou "soft")
    :lr,rf,xgb: int
    :titre: str
    :tfidf:booleen
    :return: fct predire()
    
     Création du modèle voting classifier avec ses poids lr, rf, gnb, xgb et param"""
    if tfidf:
        train_X=X_tfidf 
        test_X=X_test_tfidf
    else :
        train_X=X
        test_X=X_test
        
    eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2),('gnb', clf3), ('xgb', clf4)], voting=param, weights=[lr,rf,gnb, xgb]) 
    eclf1 = eclf1.fit(train_X, y)
    return predire(eclf1, titre, tfidf)

In [ ]:
pred_voting_hard=voting_classifier("hard",titre="Voting hard")
pred_voting_hard_tfidf=voting_classifier("hard",titre="voting hard tfidf",tfidf=True)

compare_model.append(pred_voting_hard)
compare_model.append(pred_voting_hard_tfidf)

In [ ]:
pred_voting_soft=voting_classifier("soft",titre="voting soft")
pred_voting_soft_tfidf=voting_classifier("soft", titre="voting soft tfidf",tfidf=True)

compare_model.append(pred_voting_soft)
compare_model.append(pred_voting_soft_tfidf)

In [ ]:
pred_soft_1_5rfxgb=voting_classifier("soft",titre="voting soft, poids random forest=1.5 xgboost=1.5",rf=1.5, xgb=1.5)
pred_soft_1_5rfxgb_tfidf=voting_classifier("soft", titre="voting soft, poids random forest=1.5 xgboost=1.5 tfidf",rf=1.5, xgb=1.5, tfidf=True)

compare_model.append(pred_soft_1_5rfxgb)
compare_model.append(pred_soft_1_5rfxgb_tfidf)

In [ ]:
pred_hard_1_5rfxgb=voting_classifier("hard",titre="voting hard, poids random forest=1.5 xgboost=1.5",rf=1.5, xgb=1.5)
pred_hard_1_5rfxgb_tfidf=voting_classifier("hard", titre="voting hard, poids random forest=1.5 xgboost=1.5 tfidf",rf=1.5, xgb=1.5, tfidf=True)

compare_model.append(pred_hard_1_5rfxgb)
compare_model.append(pred_hard_1_5rfxgb_tfidf)

# **V-Resultats**
#### On réccupère les modèles de calcul ayant le meilleur score.

In [ ]:
best_score=np.max(np.array([compare_model[i]["score"] for i in range(len(compare_model))]))
meilleurs_models=[]
for i in range(len(compare_model)):
    if compare_model[i]["score"]==best_score :
        meilleurs_models.append(compare_model[i])
for i in range(len(meilleurs_models)):
    print(meilleurs_models[i]["model"][:200],"\n tfidf:",meilleurs_models[i]["tfidf"],"\n")

In [ ]:
print(meilleurs_models[0])

#### On utilise les résultats du premier modèle ayant le meilleur score.


In [ ]:
output = pd.DataFrame({'id': test_df.id, 'target': meilleurs_models[0]["predictions"]})
output.to_csv('my_submission.csv', index=False)
submition_df=pd.read_csv('my_submission.csv')
submition_df.head()